In [1]:
import numpy as np
import cv2 as cv
import va

In [15]:
video = cv.VideoCapture("./simpsons abbey road.mp4")
fm = []
thr = 50
prev = None

while True:
    ret, frame = video.read()
    if not ret:
        # No more frames
        break
    f = cv.cvtColor(frame, cv.COLOR_BGR2GRAY).astype(np.uint16)
    if prev is not None:
        mask = np.zeros_like(f)
        mask[np.abs(f - prev) > thr] = 255
        fm.append((frame, mask))
    prev = f
height,width,layers=fm[0][0].shape
video.release()

output=cv.VideoWriter('./video.avi', cv.VideoWriter_fourcc(*'mp4v') ,1,(width,height))
for _, mask in fm:
    output.write(mask)
output.release()    


In [4]:
mog = cv.createBackgroundSubtractorMOG2(detectShadows=False)

video = cv.VideoCapture("./simpsons abbey road.mp4")
fm = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    f = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    mask = mog.apply(f)
    fm.append((frame, mask))
video.release()